In [2]:
pip install pretty_midi

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import librosa
import os
from matplotlib import pyplot as plt
import pretty_midi 
pretty_midi.pretty_midi.MAX_TICK = 1e10
import pandas as pd

In [4]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [12]:
#Train
import os
import librosa
import pretty_midi
import numpy as np

#preparing the data from the original audio files
entries = os.scandir('F:/meastro-v2/2004')#loading the data to process which we are going to use for training

for file in entries:
    file_name,file_extensions=os.path.splitext(file)
    if file_extensions=='.wav':
        sr=16000
        y, sr = librosa.load(file, sr=sr)
        hop_length=512
        n_fft=2048
        MFCC=librosa.feature.mfcc(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)#using the librosa library to calculate the mel of the audio file which is returned as a 2D vector 
        c=len(y) # it is being stored because it will be used for the frequency while making the piano roll file
        MFCC=np.abs(MFCC)# the returned 2d vector is a complex quantity 
        MFCC=librosa.amplitude_to_db(MFCC)# taking the amplitude on logarithmic scale gives a better visualization of data
        b=np.shape(MFCC)[1]
    
        entries1 = os.scandir('F:/meastro-v2/2004')#it is being loaded again so that we can get the corresponding midi file to the wav file which we have just used
        for file1 in entries1:
            file_name1,file_extensions1=os.path.splitext(file1)

            if file_name1==file_name and file_extensions1==".midi":
                file2=file_name + file_extensions1
                pm = pretty_midi.PrettyMIDI(file2)
                a=pm.instruments[0].get_piano_roll(float(sr*float(b)/float(c)))# The Sampling frequency has been taken such that its shape and the mel files shape is same 
                a = (a[21:109, :b] > 0)#we only need the frequency of the piano and converting non binary data to 0 and 1
                a=np.transpose(a)
                out = 'F:/meastro-v2/trainmel/melmid/mid_'+str(file_name1[40:])+'midi'+'.npy'
                np.save(out,a)
                out = 'F:/meastro-v2/trainmel/melwav/wav_'+str(file_name[40:])+'.npy'
                MFCC=np.transpose(MFCC)
                if a.shape[0]<MFCC.shape[0]:#cutting out little extra part in MFCC arrays
                    MFCC=MFCC[:a.shape[0],:]
                    print(np.shape(MFCC))
                    np.save(out,MFCC)
                    print('saving',out)


(30282, 13)
saving F:/meastro-v2/trainmel/melwav/wav_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.npy
(8357, 13)
saving F:/meastro-v2/trainmel/melwav/wav_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_06_Track06_wav.npy
(6066, 13)
saving F:/meastro-v2/trainmel/melwav/wav_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_08_Track08_wav.npy
(9770, 13)
saving F:/meastro-v2/trainmel/melwav/wav_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_10_Track10_wav.npy
(44042, 13)
saving F:/meastro-v2/trainmel/melwav/wav_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_02_Track02_wav.npy
(10240, 13)
saving F:/meastro-v2/trainmel/melwav/wav_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_03_Track03_wav.npy
(19424, 13)
saving F:/meastro-v2/trainmel/melwav/wav_05_R1_2004_02-03_ORIG_MID--AUDIO_05_R1_2004_06_Track06_wav.npy
(14928, 13)
saving F:/meastro-v2/trainmel/melwav/wav_07_R1_2004_01_ORIG_MID--AUDIO_07_R1_2004_02_Track02_wav.npy
(31100, 13)
saving F:/meastro-v2/trainmel/melwav/wav_07_R1_2004_01_ORIG_MID--AUDIO_0

In [19]:
import os
import numpy as np

def concatenate_wav(path):
    entries = os.scandir(path)
    d = np.zeros((0,100,13))
    
    for file in entries:
        if file.name.endswith('.npy'):
            a = np.load(file, allow_pickle=True)
        elif file.name.endswith('.npz'):
            a = np.load(file)['arr_0']
        else:
            continue  # skip non-numpy files
        
        a = a[np.newaxis,:,:]
        for b in range(int(np.shape(a)[1]/100)):
            c = a[:,b*100:(b*100)+100,:]
            d = np.concatenate((d,c), axis=0)   

    out = os.path.join(path, 'concatenate_wav.npy')
    d = d[:,:,:]
    np.save(out, d)
    print('saving',out)

def concatenate_mid(path):
    entries = os.scandir(path)
    d = np.zeros((0,100,88))
    
    for file in entries:
        if file.name.endswith('.npy'):
            a = np.load(file, allow_pickle=True)
        elif file.name.endswith('.npz'):
            a = np.load(file)['arr_0']
        else:
            continue  # skip non-numpy files
        
        a = a[np.newaxis,:,:]
        for b in range(int(np.shape(a)[1]/100)):
            c = a[:,b*100:(b*100)+100,:]
            d = np.concatenate((d,c), axis=0)   

    out = os.path.join(path, 'concatenate_mid.npy')
    d = d[:,:,:]
    np.save(out, d)
    print('saving',out)

path1 = 'F:/meastro-v2/trainmel/melwav'
concatenate_wav(path1)

path2 = 'F:/meastro-v2/trainmel/melmid'
concatenate_mid(path2)


saving F:/meastro-v2/trainmel/melwav\concatenate_wav.npy
saving F:/meastro-v2/trainmel/melmid\concatenate_mid.npy


In [20]:
#Train
import os
import librosa
import pretty_midi
import numpy as np

#preparing the data from the original audio files
entries = os.scandir('F:/meastro-v2/2006')#loading the data to process which we are going to use for training

for file in entries:
    file_name,file_extensions=os.path.splitext(file)
    if file_extensions=='.wav':
        sr=16000
        y, sr = librosa.load(file, sr=sr)
        hop_length=512
        n_fft=2048
        MFCC=librosa.feature.mfcc(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)#using the librosa library to calculate the mel of the audio file which is returned as a 2D vector 
        c=len(y) # it is being stored because it will be used for the frequency while making the piano roll file
        MFCC=np.abs(MFCC)# the returned 2d vector is a complex quantity 
        MFCC=librosa.amplitude_to_db(MFCC)# taking the amplitude on logarithmic scale gives a better visualization of data
        b=np.shape(MFCC)[1]
    
        entries1 = os.scandir('F:/meastro-v2/2006')#it is being loaded again so that we can get the corresponding midi file to the wav file which we have just used
        for file1 in entries1:
            file_name1,file_extensions1=os.path.splitext(file1)

            if file_name1==file_name and file_extensions1==".midi":
                file2=file_name + file_extensions1
                pm = pretty_midi.PrettyMIDI(file2)
                a=pm.instruments[0].get_piano_roll(float(sr*float(b)/float(c)))# The Sampling frequency has been taken such that its shape and the mel files shape is same 
                a = (a[21:109, :b] > 0)#we only need the frequency of the piano and converting non binary data to 0 and 1
                a=np.transpose(a)
                out = 'F:/meastro-v2/trainmel1/melmid/mid_'+str(file_name1[40:])+'midi'+'.npy'
                np.save(out,a)
                out = 'F:/meastro-v2/trainmel1/melwav/wav_'+str(file_name[40:])+'.npy'
                MFCC=np.transpose(MFCC)
                if a.shape[0]<MFCC.shape[0]:#cutting out little extra part in MFCC arrays
                    MFCC=MFCC[:a.shape[0],:]
                    print(np.shape(MFCC))
                    np.save(out,MFCC)
                    print('saving',out)


(56170, 13)
saving F:/meastro-v2/trainmel1/melwav/wav_1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav.npy
(11716, 13)
saving F:/meastro-v2/trainmel1/melwav/wav_1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_02_Track02_wav.npy
(30804, 13)
saving F:/meastro-v2/trainmel1/melwav/wav_1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_03_Track03_wav.npy
(14991, 13)
saving F:/meastro-v2/trainmel1/melwav/wav_1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_04_Track04_wav.npy
(13586, 13)
saving F:/meastro-v2/trainmel1/melwav/wav_1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_05_Track05_wav.npy
(20649, 13)
saving F:/meastro-v2/trainmel1/melwav/wav_2_2006_01_ORIG_MID--AUDIO_01_R2_2006_01_Track01_wav.npy
(28056, 13)
saving F:/meastro-v2/trainmel1/melwav/wav_2_2006_01_ORIG_MID--AUDIO_01_R2_2006_03_Track03_wav.npy
(30022, 13)
saving F:/meastro-v2/trainmel1/melwav/wav_1_2006_01-04_ORIG_MID--AUDIO_02_R1_2006_01_Track01_wav.npy
(42431, 13)
saving F:/meastro-v2/trainmel1/melwav/wav_1_2006_01-04_ORIG_MID--AUDIO_02_R1_2006_02_Track

In [21]:
import os
import numpy as np

def concatenate_wav(path):
    entries = os.scandir(path)
    d = np.zeros((0,100,13))
    
    for file in entries:
        if file.name.endswith('.npy'):
            a = np.load(file, allow_pickle=True)
        elif file.name.endswith('.npz'):
            a = np.load(file)['arr_0']
        else:
            continue  # skip non-numpy files
        
        a = a[np.newaxis,:,:]
        for b in range(int(np.shape(a)[1]/100)):
            c = a[:,b*100:(b*100)+100,:]
            d = np.concatenate((d,c), axis=0)   

    out = os.path.join(path, 'concatenate_wav.npy')
    d = d[:,:,:]
    np.save(out, d)
    print('saving',out)

def concatenate_mid(path):
    entries = os.scandir(path)
    d = np.zeros((0,100,88))
    
    for file in entries:
        if file.name.endswith('.npy'):
            a = np.load(file, allow_pickle=True)
        elif file.name.endswith('.npz'):
            a = np.load(file)['arr_0']
        else:
            continue  # skip non-numpy files
        
        a = a[np.newaxis,:,:]
        for b in range(int(np.shape(a)[1]/100)):
            c = a[:,b*100:(b*100)+100,:]
            d = np.concatenate((d,c), axis=0)   

    out = os.path.join(path, 'concatenate_mid.npy')
    d = d[:,:,:]
    np.save(out, d)
    print('saving',out)

path1 = 'F:/meastro-v2/trainmel1/melwav'
concatenate_wav(path1)

path2 = 'F:/meastro-v2/trainmel1/melmid'
concatenate_mid(path2)


saving F:/meastro-v2/trainmel1/melwav\concatenate_wav.npy
saving F:/meastro-v2/trainmel1/melmid\concatenate_mid.npy


In [28]:
#from pydub import AudioSegment
#import os

#mp3_dir = 'F:/meastro-v2/2006/'
#wav_dir = 'F:/meastro-v2/2006/'

#create new directory for WAV files if it does not exist
#if not os.path.exists(wav_dir):
#    os.makedirs(wav_dir)

#iterate over all MP3 files in directory and convert to WAV
#for filename in os.listdir(mp3_dir):
#    if filename.endswith('.mp3'):
#        # load MP3 file
#        mp3_path = os.path.join(mp3_dir, filename)
#        sound = AudioSegment.from_file(mp3_path, format='mp3')
        
        # create new filename for WAV file
#        wav_filename = os.path.splitext(filename)[0] + '.wav'
#        wav_path = os.path.join(wav_dir, wav_filename)
        
        # export WAV file
 #       sound.export(wav_path, format='wav')


In [12]:
#import os
#from pydub import AudioSegment
#import numpy as np

#def concatenate_wav(path):
 #   entries = os.scandir(path)
  #  for file in entries:
   #     if file.name.endswith('.mp3'):
    #        try:
     #           sound = AudioSegment.from_file(file.path, format='mp3')
      #          new_path = os.path.splitext(file.path)[0] + '.wav'
       #         sound.export(new_path, format='wav')
        #    except FileNotFoundError as e:
         #       print(f"Error: {e}. Skipping file {file.path}")

#def concatenate_mid(path):
 #   entries = os.scandir(path)
  #  d = []
   # for file in entries:
    #    if file.name.endswith('.npy'):
     #       data = np.load(file.path)
      #      d.append(data)
    #out = os.path.join(path, 'mid_all.npy')
    #d = np.concatenate(d, axis=0)
    #np.save(out, d)

#path1 = 'F:/meastro-v2/trainmel/melwav/'
#concatenate_wav(path1)

#path2 = 'F:/meastro-v2/trainmel/melmid/'
#concatenate_mid(path2)


In [18]:
#preparing the data from the original audio files
#entries = os.scandir('F:/meastro-v2/2006')#loading the data to process which we are going to use for training 

#for file in entries:
#    file_name,file_extensions=os.path.splitext(file)

#   if file_extensions=='.wav':
#        sr=16000
#        y, sr = librosa.load(file, sr=sr)
#        hop_length=512
#        n_fft=2048
#        MFCC=librosa.feature.mfcc(y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)#using the librosa library to calculate the mel of the audio file which is returned as a 2D vector 
#        c=len(y) # it is being stored because it will be used for the frequency while making the piano roll file
#        MFCC=np.abs(MFCC)# the returned 2d vector is a complex quantity 
#        MFCC=librosa.amplitude_to_db(MFCC)# taking the amplitude on logarithmic scale gives a better visualization of data
#        b=np.shape(MFCC)[1]
        
#        entries1 = os.scandir('F:/meastro-v2/2006')#it is being loaded again so that we can get the corresponding midi file to the wav file which we have just used
#        for file1 in entries1:
#          file_name1,file_extensions1=os.path.splitext(file1)

#          if file_name1==file_name and file_extensions1==".midi":
#            file2=file_name + file_extensions1
#            pm = pretty_midi.PrettyMIDI(file2)
#            a=pm.instruments[0].get_piano_roll(float(sr*float(b)/float(c)))# The Sampling frequency has been taken such that its shape and the mel files shape is same 
#            a = (a[21:109, :b] > 0)#we only need the frequency of the piano and converting non binary data to 0 and 1
#            a=np.transpose(a)
#            out = 'F:/meastro-v2/testmel/melmid/mid_'+str(file_name1[40:])+'midi'+'.npy'
#            np.save(out,a)
#            out = 'F:/meastro-v2/testmel/melwav/wav_'+str(file_name[40:])+'.npy'
#            MFCC=np.transpose(MFCC)
#            if a.shape[0]<MFCC.shape[0]:#cutting out little extra part in MFCC arrays
#              MFCC=MFCC[:a.shape[0],:]
#              print(np.shape(MFCC))
#            np.save(out,MFCC)
#            print('saving',out)
            


In [ ]:
import os
import soundfile as sf
import numpy as np

def concatenate_wav(path):
    entries = os.listdir(path)
    d = np.zeros((0, 100 * 44100))  # Changed the shape

    for file in entries:
        a, sr = sf.read(os.path.join(path, file))
        a = np.squeeze(a)
        a = a[:int(sr*100)]

        for b in range(int(np.shape(a)[0] / (sr*100))):
            c = a[b*sr*100:(b*sr*100) + sr*100]
            c = c[np.newaxis, :]
            d = np.concatenate((d, c), axis=0)

    d = d[:, np.newaxis, :]  # Add the new axis here
    out = os.path.join(path, 'concatenatewav.npy')
    np.save(out, d)
    print("Concatenating wav files is done.")
    
    
def concatenate_mid(path):
    entries = os.listdir(path)
    d = np.zeros((0, 100 * 88))  # Changed the shape

    for file in entries:
        a = np.load(os.path.join(path, file), allow_pickle=True)

        for b in range(int(np.shape(a)[1] / 100)):
            c = a[b * 100:(b * 100) + 100, :]
            c = c[np.newaxis, :]
            d = np.concatenate((d, c), axis=0)

    d = d[:, np.newaxis, :]  # Add the new axis here
    out = os.path.join(path, 'concatenatemidi.npy')
    np.save(out, d)
    print("Concatenating midi files is done.")
    

path1 = 'F:/meastro-v2/trainmel/melwav'
concatenate_wav(path1)

path2 = 'F:/meastro-v2/trainmel/melmid'
concatenate_mid(path2)